In [1]:
import datasets
import pandas as pd

dataset = datasets.load_dataset("trl-lib/tldr")

# Convert the dataset to a pandas DataFrame
df_train = pd.DataFrame(dataset['train'])
df_val = pd.DataFrame(dataset['validation'])

print(df_train.head())
print(df_val.head())

/workspace/hello-rl/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                              prompt  \
0  SUBREDDIT: r/relationships\n\nTITLE: I (f/22) ...   
1  SUBREDDIT: r/loseit\n\nTITLE: SV & NSV! Keepin...   
2  SUBREDDIT: r/relationships\n\nTITLE: Me [19F] ...   
3  SUBREDDIT: r/personalfinance\n\nTITLE: Priorit...   
4  SUBREDDIT: r/relationships\n\nTITLE: My[25m] g...   

                                          completion  
0   I still have contact with an old ex's friends...  
1   Progress is still happening, even when you th...  
2   My skin is scarred badly; what could I do/say...  
3   $14k in student debt (all <5%) and need to sa...  
4   GF is a meanie-bo-beanie when I'm nice, and a...  
                                              prompt  \
0  SUBREDDIT: r/AskReddit\n\nTITLE: How do you ge...   
1  SUBREDDIT: r/pettyrevenge\n\nTITLE: So, my mom...   
2  SUBREDDIT: r/relationships\n\nTITLE: My girlfr...   
3  SUBREDDIT: r/running\n\nTITLE: One year post-p...   
4  SUBREDDIT: r/tifu\n\nTITLE: TIFU by getting sc... 

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [ ]:
def tokenize_chat(row):
    chat = [
        {"role": "system", "content": row["prompt"]},
        {"role": "assistant", "content": row["completion"]},
    ]
    message = tokenizer.apply_chat_template(chat, tokenize=True)
    return message

trainset = [tokenize_chat(row) for i, row in df_train.iterrows()]
valset = [tokenize_chat(row) for i, row in df_val.iterrows()]

trainset